In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tkinter as tk
from tkinter import messagebox
import threading
import time


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Initialize Mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# State variables
mode = 0
collecting_data = False
data_collection = []

def switch_mode():
    global mode, collecting_data
    mode += 1
    collecting_data = False
    print(f"Switched to mode {mode}")

def start_collecting_data():
    global collecting_data
    collecting_data = True
    print(f"Started collecting data in mode {mode}")

# Open the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            if collecting_data:
                # Collect data (example: coordinates of landmarks)
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                data_collection.append((mode, landmarks))

    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    key = cv2.waitKey(1)
    if key == ord('1'):
        start_collecting_data()
    elif key == ord('2'):
        switch_mode()
    elif key == ord('3'):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()

# Save the collected data for further use
with open('hand_gesture_data.pkl', 'wb') as f:
    pickle.dump(data_collection, f)


Started collecting data in mode 0
Switched to mode 1
Started collecting data in mode 1
Switched to mode 2
Started collecting data in mode 2
Switched to mode 3
Started collecting data in mode 3
Switched to mode 4
Started collecting data in mode 4
Switched to mode 5
Started collecting data in mode 5
Switched to mode 6
Started collecting data in mode 6
Switched to mode 7
Started collecting data in mode 7
Switched to mode 8
Started collecting data in mode 8
Switched to mode 9


In [ ]:
# Load the collected data
with open('hand_gesture_data.pkl', 'rb') as f:
    data_collection = pickle.load(f)

# Prepare the data
X = []
y = []
for mode, landmarks in data_collection:
    flattened_landmarks = [coord for point in landmarks for coord in point]
    X.append(flattened_landmarks)
    y.append(mode)

X = np.array(X)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(63,)),  # 21 landmarks * 3 coordinates
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Save the trained model
model.save('gesture_recognition_model.h5')


Epoch 1/20


c:\Users\ahmac\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3894 - loss: 1.9456 - val_accuracy: 0.9196 - val_loss: 0.8879
Epoch 2/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9145 - loss: 0.6476 - val_accuracy: 0.9479 - val_loss: 0.2683
Epoch 3/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9597 - loss: 0.2202 - val_accuracy: 0.9989 - val_loss: 0.1308
Epoch 4/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9879 - loss: 0.1243 - val_accuracy: 0.9921 - val_loss: 0.0806
Epoch 5/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9980 - loss: 0.0696 - val_accuracy: 1.0000 - val_loss: 0.0409
Epoch 6/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9997 - loss: 0.0402 - val_accuracy: 1.0000 - val_loss: 0.0255
Epoch 7/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0243 - val_accuracy: 1.0000 - val_loss: 0.0169
Epoch 8/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0160 - val_accuracy: 1.0000 - val_

In [ ]:

# Load the trained model
model = load_model('gesture_recognition_model.h5')

# Initialize Mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# State variables
mode = 0

def switch_mode():
    global mode
    mode += 1
    print(f"Switched to mode {mode}")

# Open the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Prepare the landmarks for prediction
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            flattened_landmarks = np.array([coord for point in landmarks for coord in point]).reshape(1, -1)

            # Predict the gesture
            prediction = model.predict(flattened_landmarks)
            predicted_mode = np.argmax(prediction)

            cv2.putText(frame, f'Mode: {predicted_mode}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    key = cv2.waitKey(1)
    if key == ord('2'):
        switch_mode()
    elif key == ord('3'):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [ ]:
import tkinter as tk

class RestaurantMenu:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Restaurant Menu")
        self.root.geometry("600x400")
        self.root.configure(bg="blue")

        self.current_menu = "main"

        self.main_menu()

    def main_menu(self):
        self.clear_screen()

        # Title
        title_label = tk.Label(self.root, text="Welcome", bg="blue", fg="white", font=("Helvetica", 30, "bold"))
        title_label.pack(pady=20)

        # Restaurant name and logo
        name_label = tk.Label(self.root, text="Restaurant Name", bg="blue", fg="white", font=("Helvetica", 14))
        name_label.place(x=10, y=10)

        # Menu options
        food_button = tk.Button(self.root, text="Food Selection", command=self.food_menu, font=("Helvetica", 14))
        food_button.pack(pady=10)

        chef_button = tk.Button(self.root, text="Chef Selection", command=self.chef_menu, font=("Helvetica", 14))
        chef_button.pack(pady=10)

    def food_menu(self):
        self.current_menu = "food"
        self.clear_screen()

        # Add photo and price here

        # Sample dishes
        dishes = [
            ("Dish 1", "$10"),
            ("Dish 2", "$12"),
            ("Dish 3", "$15"),
            ("Dish 4", "$8"),
            ("Dish 5", "$11"),
            ("Dish 6", "$9"),
            ("Dish 7", "$13"),
            ("Dish 8", "$14"),
            ("Dish 9", "$7"),
            ("Dish 10", "$16")
        ]

        # Display first five dishes
        for i in range(5):
            dish_label = tk.Label(self.root, text=dishes[i][0] + " - " + dishes[i][1], bg="blue", fg="white", font=("Helvetica", 14))
            dish_label.pack(pady=5)

        # More Foods option
        more_foods_button = tk.Button(self.root, text="More Foods", command=lambda: self.show_more_foods(dishes), font=("Helvetica", 14))
        more_foods_button.pack(pady=10)

        # Back button
        back_button = tk.Button(self.root, text="Back", command=self.main_menu, font=("Helvetica", 14))
        back_button.pack(pady=10)

    def show_more_foods(self, dishes):
        self.clear_screen()

        # Display next five dishes
        for i in range(5, 10):
            dish_label = tk.Label(self.root, text=dishes[i][0] + " - " + dishes[i][1], bg="blue", fg="white", font=("Helvetica", 14))
            dish_label.pack(pady=5)

        # Back button
        back_button = tk.Button(self.root, text="Back", command=self.food_menu, font=("Helvetica", 14))
        back_button.pack(pady=10)

    def chef_menu(self):
        self.current_menu = "chef"
        self.clear_screen()

        # Display head chef's selection
        head_chef_selection = "Chef's Selection: Dish XYZ"
        chef_label = tk.Label(self.root, text=head_chef_selection, bg="blue", fg="white", font=("Helvetica", 18))
        chef_label.pack(pady=20)

        # Back button
        back_button = tk.Button(self.root, text="Back", command=self.main_menu, font=("Helvetica", 14))
        back_button.pack(pady=10)

    def clear_screen(self):
        # Clear previous screen
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    menu = RestaurantMenu()
    menu.root.mainloop()




In [ ]:


# Load the trained model
model = load_model('gesture_recognition_model.h5')

# Initialize Mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# Menu data
dishes = [
    ("Spaghetti", 10.99),
    ("Burger", 8.99),
    ("Salad", 6.99),
    ("Pizza", 11.99),
    ("Sushi", 14.99)
]

# Initialize tkinter GUI
root = tk.Tk()
root.title("Restaurant Menu")
root.geometry("400x300")
root.configure(bg="blue")

menu_frame = None

current_mode = 0
current_selection = 0
current_frame = None

def switch_mode(new_mode):
    global current_mode, current_selection, current_frame
    current_mode = new_mode
    
    if current_mode == 0:
        show_order_menu()
    elif current_mode == 0:
        show_chef_suggestion()
    elif current_mode == 8:
        reset_to_main_menu()
    elif current_mode in [2, 3]:  # Navigation
        if current_frame == 'menu':
            navigate_main_menu()
        elif current_frame == 'order':
            navigate_order_menu()

def delayed_action(predicted_mode):
    # Wait for 2 seconds
    threading.Timer(5.0, switch_mode, args=(predicted_mode,)).start()

# Usage
predicted_mode = 2  # یا هر مقدار دیگری که بخواهید
delayed_action(predicted_mode)

def show_order_menu():
    global current_frame
    current_frame = 'order'
    clear_frame()
    order_frame = tk.Frame(root, bg="blue")
    order_frame.pack(expand=True)
    order_label = tk.Label(order_frame, text="Select a Dish", font=("Arial", 24), bg="blue", fg="white")
    order_label.pack(pady=10)
    
    for idx, (dish, price) in enumerate(dishes):
        dish_button = tk.Button(order_frame, text=f"{idx+1}. {dish} - ${price:.2f}", font=("Arial", 18),
                                command=lambda idx=idx: confirm_order(idx))
        dish_button.pack(pady=5)

    back_button = tk.Button(order_frame, text="Return", font=("Arial", 18), command=lambda: switch_mode(8))
    back_button.pack(pady=10)

def show_chef_suggestion():
    global current_frame
    current_frame = 'chef_suggestion'
    clear_frame()
    suggestion_frame = tk.Frame(root, bg="blue")
    suggestion_frame.pack(expand=True)
    
    suggestion = dishes[0]  # Simple suggestion logic, can be randomized or more complex
    suggestion_label = tk.Label(suggestion_frame, text=f"Chef's Suggestion:\n{suggestion[0]} - ${suggestion[1]:.2f}",
                                font=("Arial", 24), bg="blue", fg="white")
    suggestion_label.pack(pady=20)
    
    back_button = tk.Button(suggestion_frame, text="Return", font=("Arial", 18), command=lambda: switch_mode(8))
    back_button.pack(pady=10)

def confirm_order(dish_idx):
    dish = dishes[dish_idx]
    response = messagebox.askyesno("Confirm Order", f"Do you want to order {dish[0]} for ${dish[1]:.2f}?")
    if response:
        messagebox.showinfo("Order Confirmed", f"You have ordered {dish[0]}")
        switch_mode(8)  # Return to main menu

def reset_to_main_menu():
    global current_frame
    current_frame = 'menu'
    clear_frame()
    create_main_menu()

def create_main_menu():
    global menu_frame, current_selection
    current_selection = 0  # Reset selection
    menu_frame = tk.Frame(root, bg="blue")
    menu_frame.pack(expand=True)

    menu_label = tk.Label(menu_frame, text="Restaurant Menu", font=("Arial", 24), bg="blue", fg="white")
    menu_label.pack(pady=10)

    option1_button = tk.Button(menu_frame, text="1. Order Food", font=("Arial", 18), command=lambda: switch_mode(0))
    option1_button.pack(pady=10)

    option2_button = tk.Button(menu_frame, text="2. Chef's Suggestion", font=("Arial", 18), command=lambda: switch_mode(0))
    option2_button.pack(pady=10)

def clear_frame():
    for widget in root.winfo_children():
        widget.destroy()

def navigate_main_menu():
    global current_selection
    if current_mode == 2:  # Go up
        current_selection = (current_selection - 1) % 2
    elif current_mode == 3:  # Go down
        current_selection = (current_selection + 1) % 2
    highlight_main_menu_selection()

def navigate_order_menu():
    global current_selection
    if current_mode == 2:  # Go up
        current_selection = (current_selection - 1) % len(dishes)
    elif current_mode == 3:  # Go down
        current_selection = (current_selection + 1) % len(dishes)
    highlight_order_menu_selection()

def highlight_main_menu_selection():
    for idx, widget in enumerate(menu_frame.winfo_children()):
        widget.configure(bg="blue")
        if isinstance(widget, tk.Button) and idx == current_selection + 1:  # +1 to skip the title label
            widget.configure(bg="white")

def highlight_order_menu_selection():
    order_frame = root.winfo_children()[0]
    for idx, widget in enumerate(order_frame.winfo_children()):
        widget.configure(bg="blue")
        if isinstance(widget, tk.Button) and idx == current_selection + 1:  # +1 to skip the title label
            widget.configure(bg="white")

def capture_gesture():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        root.after(10, capture_gesture)  # Retry after 10ms
        return

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Prepare the landmarks for prediction
            landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
            flattened_landmarks = np.array([coord for point in landmarks for coord in point]).reshape(1, -1)

            # Predict the gesture
            prediction = model.predict(flattened_landmarks)
            predicted_mode = np.argmax(prediction)

            if predicted_mode in [0, 1,2,3 ,4, 5, 6, 7, 2, 3, 8]:  # Relevant modes
                switch_mode(predicted_mode)
                cv2.putText(frame, f'Mode: {predicted_mode}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Hand Gesture Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('3'):
        root.quit()  # Quit the application if 'q' is pressed
    else:
        root.after(10, capture_gesture)  # Call this function again after 10ms

# Open the webcam
cap = cv2.VideoCapture(0)

# Create main menu initially
create_main_menu()

# Start capturing gestures
root.after(5, capture_gesture)

# Start the tkinter main loop
root.mainloop()

# Release the resources after the loop ends
cap.release()
cv2.destroyAllWindows()


c:\Users\ahmac\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━